In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 500)
pd.set_option("display.min_rows", 200)
pd.set_option('display.max_colwidth', None)

pd.options.mode.chained_assignment = None

import sys
sys.path.append('../../..')

from src.box import read_box_df

In [124]:
ddf = pd.read_csv('../data/indiana/indiana_guardianship_dockets.csv')
cdf = pd.read_csv('../data/indiana/indiana_clean_agc_filings.csv')
rdf = pd.read_csv('../data/indiana/indiana_agc_results.csv')

cdf['case_number'] = cdf['case_number'].str.upper()
ddf['case_number'] = ddf['case_number'].str.upper()
rdf_m = cdf.merge(rdf, on='case_number', how='left')

missing_results_casenums = rdf_m[~rdf_m['Unnamed: 0'].isnull() & rdf_m['court'].isnull()]['case_number'].unique().tolist()

print('Number of cases in results scrape:', rdf.shape[0])
print('Number of cases not in registry:', rdf.court.isnull().sum())
print('Percentage of missing cases:', rdf.court.isnull().sum()/rdf.shape[0]*100)
print('Number of cases not in results scrape:', rdf_m['Unnamed: 0'].isnull().sum())

print('Types of guardianship counts:', rdf.guardianship_type.value_counts())

def clean_text(df, colname):
    space_replace_list = ['\n', '\/']
    for sp in space_replace_list:
        df[colname] = df[colname].str.replace(sp, ' ', regex=True)
    df[colname] = df[colname].str.replace("[^A-Za-z ]", '', regex=True)
    df[colname] = df[colname].str.replace("\s+", ' ', regex=True)
    df[colname] = df[colname].str.replace(" co\s+", ' co', regex=True)
    return df[colname].str.strip().str.lower()

ddf['combined_docket_event_description'] = ddf['docket_event_description'].fillna('') + ' ' + ddf['docket_event_comment'].fillna('')
ddf['combined_docket_event_description'] = clean_text(ddf, 'combined_docket_event_description')

# flag explicit mention of temporary or permanent guardian
ddf['temp_guardian_mentioned'] = ddf.combined_docket_event_description.str.contains('temporary guardian')
ddf['perm_guardian_mentioned'] = ddf.combined_docket_event_description.str.contains('permanent guardian')


vardf = pd.read_csv('../data/indiana/indiana_2024_variables.csv')
vardf['value'] = clean_text(vardf, 'value')
var_dicts = vardf.to_dict('records')
var_cols = vardf[~vardf.value.isnull()].varname.unique().tolist()

# FILTER DDF TO ONLY INCLUDE CASES MISSING REGISTRY
ddf_m = ddf[ddf.case_number.isin(cases_not_found_in_registry)]

Number of cases in results scrape: 7556
Number of cases not in registry: 1882
Percentage of missing cases: 24.9073583906829
Number of cases not in results scrape: 2233
Types of guardianship counts: guardianship_type
Permanent    5086
Temporary     588
Name: count, dtype: int64


## Results Data Scrape

In [3]:
rdf.head()

,Unnamed: 0,case_number,court,ward_name,ward_type,birth_year,guardianship_type,guardian(s),guardianship_scope,issue_date,expiration_date
0,0,82D04-1610-GU-000571,"Vanderburgh Superior Court 4, Vanderburgh County, Indiana",Christopher R. Mains,Incapacitated Adult,1975,Temporary,Danielle G. DeChristopher,Person and Estate,2/16/2017,Terminated
1,1,40C01-1702-GU-000010,"Jennings Circuit Court, Jennings County, Indiana",Barbara M. Hardin,Incapacitated Adult,1948,Permanent,Debra L. Woodard,Person and Estate,3/28/2017,Terminated
2,2,52C01-1812-GU-000017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,78A05-1609-GU-02233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,73D01-1605-GU-000036,"Shelby Superior Court 1, Shelby County, Indiana",Ezra P. Cradic,Incapacitated Adult,1933,Permanent,Kimberly A. Arnold; Kristi S. Buis; Kevin P. Cradic,Person and Estate; Person and Estate; Person and Estate,6/6/2016; 6/6/2016; 6/6/2016,Terminated; Terminated; Terminated


TODO:

* Scrape new cases from registry
    * write to ddj repo
    * create CSV with new cases that need to be scraped (new ones first and then retry old ones?)
* Look at docket entries for cases that are not in registry
* Validate docket inference with registry results
* Look at v1 Alaska dataset and pull together analogous Indiana deliverable
* QA: Why are there cases in results dataframe that are not in cases data?
* Add "temporary guardian"/"permanent_guardian" flag to see if you can use "granted" + flag to label
* Look at associated comments to "Order Granting" event

In [49]:
rdf_m[rdf_m['Unnamed: 0'].isnull()][['case_number']].to_csv('case_numbers_for_registry_scrape.csv')

### Important: Quality Assurance

In [69]:
cases_not_found_in_registry = rdf[rdf.court.isnull()].case_number.unique().tolist()
len(cases_not_found_in_registry)

1882

In [70]:
# where are there fewer cases with missing results after merge? 1882 vs 1799
# it's because there are 83 cases that are in results data but not in the case data 
(~rdf_m['Unnamed: 0'].isnull() & rdf_m['court'].isnull()).sum()

1799

In [64]:
# TODO: case numbers that are in results data but not in case dataframe
rdf[~rdf.case_number.isin(cdf.case_number.unique().tolist())]

(83, 11)

## Searching Docket Text

### Flagging Temporary/Permanent Guardian

In [145]:
# dataframe with concatenated 
granted_list = [
    "((temporary|permanent) )?guardianship ((is|was) )?granted", 
    "order.*appoint.*guardian", 
    "guardian appoint", 
    "granted ((permanent|temporary) )?guardianship",
    "letters of ((permanent|temporary) )?guardianship issued",
    "terminate guardianship",
    "successor guardian",
    "Order Granting Petition to Establish Guardianship".lower()
]
ddf_m['granted'] = False
for g in granted_list:
    query = (
        ddf_m.combined_docket_event_description.str.contains(g, regex=True)
        & ~ddf_m.combined_docket_event_description.str.contains("( not |reject|deny|denied|dismiss|ad litem|hearing)", regex=True)
        # & ~ddf_m.combined_docket_event_description.str.startswith("petition to")
    )
    print('Results:', g, query.sum())
    ddf_m.loc[query, 'granted'] = True

Results: ((temporary|permanent) )?guardianship ((is|was) )?granted 6
Results: order.*appoint.*guardian 672
Results: guardian appoint 131
Results: grant guardianship 0
Results: letters of ((permanent|temporary) )?guardianship issued 1007
Results: terminate guardianship 578
Results: successor guardian 93
Results: order granting petition to establish guardianship 698


In [130]:
ddf_m.columns

Index(['c2dp_case_id', 'fips', 'case_key', 'state', 'county', 'case_number',
       'raw_case_type', 'clean_case_type', 'date_filed', 'date_closed',
       'date_disposed', 'disposition', 'raw_case_status', 'clean_case_status',
       'is_closed', 'docket_eventkey', 'docket_casekey',
       'docket_baseeventtype', 'docket_baseeventtypekey', 'docket_eventtype',
       'docket_event_date', 'docket_event_description', 'docket_event_comment',
       'combined_docket_event_description', 'temp_guardian_mentioned',
       'perm_guardian_mentioned', 'granted'],
      dtype='object')

In [139]:
ddf_m[ddf_m.granted & ddf_m.temp_guardian_mentioned]

,c2dp_case_id,fips,case_key,state,county,case_number,raw_case_type,clean_case_type,date_filed,date_closed,date_disposed,disposition,raw_case_status,clean_case_status,is_closed,docket_eventkey,docket_casekey,docket_baseeventtype,docket_baseeventtypekey,docket_eventtype,docket_event_date,docket_event_description,docket_event_comment,combined_docket_event_description,temp_guardian_mentioned,perm_guardian_mentioned,granted
1678,49D08-2111-GU-037038,18097,indiana___49D08-2111-GU-037038,indiana,marion,49D08-2111-GU-037038,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2021-11-04,NaN,NaN,NaN,pending,open/pending,False,654032795,42249716,C,C,ADMOAF,11/04/2021,Oath Filed,Acceptance and Oath of Temporary Guardian Appointed Over Estate of Incapacitated Person,oath filed acceptance and oath of temporary guardian appointed over estate of incapacitated person,True,False,True
6515,45D11-1812-GU-000302,18089,indiana___45D11-1812-GU-000302,indiana,lake,45D11-1812-GU-000302,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2018-12-12,2019-06-25,NaN,NaN,decided,disposed/closed,True,539969490,34540548,C,ORD,OG,10/02/2019,Order Granting,Order Appointing Temporary Guardian Over the Estate of An Incapacitated Adult,order granting order appointing temporary guardian over the estate of an incapacitated adult,True,False,True
11726,02D03-1811-GU-000317,18003,indiana___02D03-1811-GU-000317,indiana,allen,02D03-1811-GU-000317,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2018-11-26,2018-11-28,NaN,NaN,decided,disposed/closed,True,482849691,34256630,C,ORD,ORD,11/27/2018,Order Issued,Order Appointing Temporary Guardianship[ over an adult,order issued order appointing temporary guardianship over an adult,True,False,True
11728,02D03-1811-GU-000317,18003,indiana___02D03-1811-GU-000317,indiana,allen,02D03-1811-GU-000317,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2018-11-26,2018-11-28,NaN,NaN,decided,disposed/closed,True,482929539,34256630,C,ORD,ORD,11/28/2018,Order Issued,Amended Order Appointing Temporary Guardian Over An Adult,order issued amended order appointing temporary guardian over an adult,True,False,True
11808,53C01-1809-GU-000118,18105,indiana___53C01-1809-GU-000118,indiana,monroe,53C01-1809-GU-000118,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2018-09-07,2021-06-30,NaN,NaN,decided,disposed/closed,True,474774360,33628228,C,ORD,OGTG,09/27/2018,Order Granting Temporary Guardianship,Order Appointing Temporary Guardian for an Incapacitated Adult issued. / CC,order granting temporary guardianship order appointing temporary guardian for an incapacitated adult issued cc,True,False,True
12652,82D04-1701-GU-000054,18163,indiana___82D04-1701-GU-000054,indiana,vanderburgh,82D04-1701-GU-000054,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2017-01-27,2017-07-31,NaN,NaN,decided,disposed/closed,True,353623184,26150561,C,ORD,OGTG,01/27/2017,Order Granting Temporary Guardianship,Order granting Emergency Motion for Appointment of Temporary Guardian having been approved. Susan Wilkie appointed as temp. gdn. over the person of Mary E. Englebright. (RAF/sg),order granting temporary guardianship order granting emergency motion for appointment of temporary guardian having been approved susan wilkie appointed as temp gdn over the person of mary e englebright raf sg,True,False,True
12656,82D04-1701-GU-000054,18163,indiana___82D04-1701-GU-000054,indiana,vanderburgh,82D04-1701-GU-000054,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2017-01-27,2017-07-31,NaN,NaN,decided,disposed/closed,True,353624190,26150561,C,OTHER,LG,01/27/2017,Letters of Guardianship Issued,Letters of Temporary Guardianship is issued. (RAF/sg),letters of guardianship issued letters of temporary guardianship is issued raf sg,True,False,True
12663,82D04-1701-GU-000054,18163,indiana___82D04-1701-GU-000054,indiana,vanderburgh,82D04-17

In [134]:
ddf_m.granted.sum()

2963

### Phrases from CSV

In [86]:
check_values = lambda val: (ddf_m.docket_event_description.str.lower() == val) | ddf_m.docket_event_comment.str.lower().str.contains(val)
check_partial_values = lambda val: (ddf_m.combined_docket_event_description.str.lower().str.contains(val, na=False))

for var in var_dicts:
    if var['varname'] and (var['value'] == var['value']):
        try:
            val = var['value'].lower().strip()
            ddf_m[var['varname']] = check_partial_values(val)
            print(var['varname'], var['value'], ddf_m[var['varname']].sum())
        except:
            print('ERROR', var['value'])

summed_cases = ddf_m.groupby('case_number')[var_cols].sum().reset_index()
cdf_merge = cdf.merge(summed_cases, on='case_number', how='right')

keyword_cols = lambda val: [c for c in cdf_merge.columns if c.startswith(val) and val + '_' not in c]

medical_ physicians report filed 511
medical1 dr evaluation 0
medical2 psycho evaluation 0
accounting_1 order approving biennial accounting 244
p1 verified petition for appointment of adult guardian efiled 0
p2 petition for appointment of a guardian conservator 0
p3 petition for appointment of coguardians over person and estate 20
p4 petition to establish guardianship 6062
p5 petitioning court for permanent guardianship 0
pcon conservatorship order for hearing 0
p_emerg petition for emergency temporary guardianship filed 441
p_emerg1 motion for interim order 1
p_emerg2 petition for emergency appointment of temporary guardian 11
p_emerg3 request for initial guardianship hearing 0
temp_order appointing temporary guardian over person estate 0
temp_order1 order granting temporary guardianship 287
temp_order2 acceptance of appointment as temporary guardian 0
temp_order3 order appointing temporary guardian of the person and conservator 0
temp_order4 letters of guardianship issued temporary a

In [87]:
cdf_merge['gc_denied'] = cdf_merge[keyword_cols('long_denied')].sum(axis='columns') > 0
print('Permanent guardianships denied:', cdf_merge['gc_denied'].sum())

cdf_merge['perm_gc_approved'] = cdf_merge[keyword_cols('long')].sum(axis='columns') > 0
print('Permanent guardianships granted:', cdf_merge['perm_gc_approved'].sum())

cdf_merge['temp_gc_approved'] = cdf_merge[keyword_cols('temp_order')].sum(axis='columns') > 0
print('Temporary guardianships granted:', cdf_merge['temp_gc_approved'].sum())

cdf_merge['limited_gc_approved'] = cdf_merge[keyword_cols('limited_order')].sum(axis='columns') > 0
print('Limited guardianships granted:', cdf_merge['limited_gc_approved'].sum())

cdf_merge['gc_dismissed'] = cdf_merge[keyword_cols('dismiss')].sum(axis='columns') > 0
print('Petition guardianship dismissed:', cdf_merge['gc_dismissed'].sum())
 
print('Overlap between granted/denied:', (cdf_merge['guardianship_denied'] & cdf_merge['perm_gc_approved']).sum())
print('Overlap between temp and perm AGC orders:', (cdf_merge['temp_gc_approved'] & cdf_merge['perm_gc_approved']).sum())

# how many neither approved or denied
outcome_cols = ['perm_gc_approved', 'temp_gc_approved', 'limited_gc_approved', 'gc_denied', 'gc_dismissed']
missing_df = cdf_merge.copy()
for c in outcome_cols:
    missing_df = missing_df[~missing_df[c]]

print('Missing disposition count:', missing_df.shape[0])

Permanent guardianships denied: 62
Permanent guardianships granted: 338
Temporary guardianships granted: 245
Limited guardianships granted: 2
Petition guardianship dismissed: 672
Overlap between granted/denied: 1
Overlap between temp and perm AGC orders: 55
Missing disposition count: 696


In [92]:
missing_disp_frequency_counts = ddf_m[ddf_m.case_number.isin(missing_df.case_number.unique().tolist())].groupby('docket_event_description')['c2dp_case_id'].count().reset_index()


In [100]:
substr = 'Guardian'
missing_disp_frequency_counts[missing_disp_frequency_counts.docket_event_description.str.contains(substr)]

,docket_event_description,c2dp_case_id
54,Expiration of Temporary Guardianship,1
66,Guardian Status Report Filed,24
67,Guardianship Information Sheet Filed,203
83,Hearing on Motion to Remove Guardian,3
84,Hearing on Motion to Terminate Guardianship,7
87,Hearing on Petition for Emergency Temporary Guardianship,38
88,Hearing on Petition to Establish Guardianship,384
93,Instruction Executed - Executor/Personal Rep/ Guardian,74
106,Letters of Guardianship Issued,166
142,Motion to Remove Guardian Filed,5


In [14]:
# cdf_merge.to_csv('../data/indiana/indiana_clean_guardianship.csv', index=False)

## Docket EDA

In [18]:
ddf.columns

Index(['c2dp_case_id', 'fips', 'case_key', 'state', 'county', 'case_number',
       'raw_case_type', 'clean_case_type', 'date_filed', 'date_closed',
       'date_disposed', 'disposition', 'raw_case_status', 'clean_case_status',
       'is_closed', 'docket_eventkey', 'docket_casekey',
       'docket_baseeventtype', 'docket_baseeventtypekey', 'docket_eventtype',
       'docket_event_date', 'docket_event_description', 'docket_event_comment',
       'combined_docket_event_description', 'medical_', 'medical1', 'medical2',
       'accounting_1', 'p1', 'p2', 'p3', 'p4', 'p5', 'pcon', 'p_emerg',
       'p_emerg1', 'p_emerg2', 'p_emerg3', 'temp_order', 'temp_order1',
       'temp_order2', 'temp_order3', 'temp_order4', 'temp_order5',
       'temp_order6', 'temp_order7', 'limited_order1:', 'limited_order2',
       'limited_order3', 'Dismiss_temp', 'long1', 'long2', 'long3', 'long5',
       'long6', 'long7', 'long8', 'long9', 'long10', 'long11', 'long12',
       'long13', 'long14', 'long15', 'long

In [19]:
NUM_OF_EXAMPLES = 10
def get_docket_entry_data(df):
    data = {}
    data['count'] = df.case_number.nunique()
    data['case_examples'] = "; ".join(df.sample(NUM_OF_EXAMPLES, replace=True).case_number.unique().tolist())
    return pd.Series(data)

frequency_table = ddf.groupby('docket_event_description').apply(get_docket_entry_data).sort_values('count', ascending=False).reset_index()
frequency_table.to_csv(f'../data/indiana/docket_keyword_search_results/all.csv')

### Comment EDA

In [20]:
dates or numbers/symbols generally
names
----

SyntaxError: invalid syntax (2495204782.py, line 1)

In [ ]:
ddf['clean_comment'] = ddf.docket_event_comment.str.lower()
print('Variations before:', ddf.clean_comment.nunique())
erase_list = ['----.*', '[\d\/\,\.]', '\: *(am|pm)', '\n']
for e in erase_list:
    ddf['clean_comment'] = ddf['clean_comment'].str.replace(e, ' ', regex=True)
print('Variations after cleaning:', ddf.clean_comment.nunique())

In [ ]:
ddf[ddf.docket_event_comment.str.contains('Acceptance of Appointment as Temporary Guardian', na=False)]

### Keyword Searches

In [21]:
ddf.head()

,c2dp_case_id,fips,case_key,state,county,case_number,raw_case_type,clean_case_type,date_filed,date_closed,date_disposed,disposition,raw_case_status,clean_case_status,is_closed,docket_eventkey,docket_casekey,docket_baseeventtype,docket_baseeventtypekey,docket_eventtype,docket_event_date,docket_event_description,docket_event_comment,combined_docket_event_description,medical_,medical1,medical2,accounting_1,p1,p2,p3,p4,p5,pcon,p_emerg,p_emerg1,p_emerg2,p_emerg3,temp_order,temp_order1,temp_order2,temp_order3,temp_order4,temp_order5,temp_order6,temp_order7,limited_order1:,limited_order2,limited_order3,Dismiss_temp,long1,long2,long3,long5,long6,long7,long8,long9,long10,long11,long12,long13,long14,long15,longcons1,long_denied,dismiss1,dismiss2,dismiss3,dismiss4,dismiss5,dismiss6,dismiss7,dismiss8,dismiss9,death,death1,death2,death3,death4,death5,death6
0,40C01-1811-GU-000046,18079,indiana___40C01-1811-GU-000046,indiana,jennings,40c01-1811-gu-000046,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2018-11-01,2019-01-15,NaN,NaN,decided,disposed/closed,True,480773008,34196766,C,C,QCSNEW,11/01/2018,Case Opened as a New Filing,NaN,case opened as a new filing,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,40C01-1811-GU-000046,18079,indiana___40C01-1811-GU-000046,indiana,jennings,40c01-1811-gu-000046,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2018-11-01,2019-01-15,NaN,NaN,decided,disposed/closed,True,480773547,34196766,C,C,APP,11/01/2018,Appearance Filed,Appearance,appearance filed appearance,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,40C01-1811-GU-000046,18079,indiana___40C01-1811-GU-000046,indiana,jennings,40c01-1811-gu-000046,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2018-11-01,2019-01-15,NaN,NaN,decided,disposed/closed,True,480773577,34196766,C,ORGFD,PEG,11/01/2018,Petition to Establish Guardianship Filed,Petition For Appointment of Co-Guardians Over Person And Estate of Incapacitated Person,petition to establish guardianship filed petition for appointment of coguardians over person and estate of incapacitated person,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,40C01-1811-GU-000046,18079,indiana___40C01-1811-GU-000046,indiana,jennings,40c01-1811-gu-000046,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2018-11-01,2019-01-15,NaN,NaN,decided,disposed/closed,True,480773583,34196766,C,C,ADMOAF,11/01/2018,Oath Filed,Oath,oath filed oath,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,40C01-1811-GU-000046,18079,indiana___40C01-1811-GU-000046,indiana,jennings,40c01-1811-gu-000046,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2018-11-01,2019-01-15,NaN,NaN,decided,disposed/closed,True,480773656,34196766,C,MOT,MHRG,11/01/2018,

Looking for "granted", "approved"

In [22]:
def search_keyword(wrds: list[str], persist=False): 
    kwdf = frequency_table
    for wd in wrds:
        kwdf = kwdf[kwdf.docket_event_description.str.lower().str.contains(wd, regex=True)]

    if persist:
        filename = "__and__".join([wrd.replace("(","").replace(")", "").replace("|", "_or_") for wrd in wrds])
        print(filename)
        kwdf.to_csv(f'../data/indiana/docket_keyword_search_results/{filename}.csv')
    return kwdf

In [23]:
search_keyword

<function __main__.search_keyword(wrds: list[str], persist=False)>

In [24]:
search_keyword(['(grant|approv|accept)'], persist=True)

grant_or_approv_or_accept


,docket_event_description,count,case_examples
9,Order Granting Petition to Establish Guardianship,6502,71j01-1701-gu-000013; 49d08-1701-gu-004003; 27d03-1811-gu-000113; 49d08-2111-gu-038255; 02d03-2106-gu-000131; 02d03-1604-gu-000082; 49d08-1706-gu-023555; 10c01-1605-gu-000083; 45c01-1809-gu-000219; 41d01-2008-gu-000131
31,Order Granting Temporary Guardianship,1829,41d01-2011-gu-000165; 49d08-1906-gu-024381; 49d08-2012-gu-045186; 45d05-2206-gu-000170; 02d03-2003-gu-000060; 49d08-1609-gu-033799; 79c01-1703-gu-000038; 49d08-2305-gu-020263; 53c01-2006-gu-000066; 53c01-2107-gu-000089
32,Order Granting Motion to Terminate Guardianship,1793,60c01-1711-gu-000029; 49d08-2001-gu-002027; 49d08-2102-gu-004946; 31c01-1604-gu-000015; 49d08-1609-gu-033865; 41d01-1707-gu-000109; 49d08-1602-gu-006113; 52d01-2002-gu-000006; 49d08-1702-gu-006582; 41d01-1611-gu-000174
35,Order Granting,1735,45d02-1809-gu-000202; 45d11-1810-gu-000234; 10c01-1712-gu-000187; 49d08-1704-gu-013845; 49d08-1607-gu-025231; 45c01-1610-gu-000161; 41d01-1610-gu-000159; 53c07-1803-gu-000033; 49d08-2207-gu-023755
36,Order Granting Motion to Withdraw Appearance,1569,53c01-1603-gu-000047; 49d08-1803-gu-011450; 02d03-1805-gu-000121; 49d08-2106-gu-021022; 13c01-1703-gu-000006; 49d08-2201-gu-003211; 10c04-1610-gu-000157; 02d03-1805-gu-000117; 49d08-1804-gu-012707; 49d08-1910-gu-045095
44,Order Granting Motion for Continuance,1204,30d01-1605-gu-000031; 73d01-1605-gu-000035; 49d08-2106-gu-021750; 24c01-1608-gu-000495; 71c01-1610-gu-000222; 53c01-1604-gu-000061; 02d03-2206-gu-000159; 49d08-1605-gu-015841; 50c01-2304-gu-000024
48,Order Approving,1028,49d08-1607-gu-025311; 32d01-1701-gu-000011; 33c01-1710-gu-000068; 20d01-2104-gu-000081; 82d04-1703-gu-000169; 73d01-1801-gu-000001; 84d02-1706-gu-004792; 49d08-1909-gu-037402; 18c01-1608-gu-000085; 32d01-1802-gu-000017
50,Order Granting Motion for Fee Waiver,935,49d08-1605-gu-017325; 52c01-2101-gu-000005; 49d08-1604-gu-014428; 10c04-2102-gu-000017; 49d08-2111-gu-038003; 49d08-2104-gu-011443; 49d08-1607-gu-026602; 02d01-1608-gu-000192; 49d08-2207-gu-023755; 49d08-1911-gu-047029
58,Order Granting Motion for Allowance of Fees,735,02d03-2103-gu-000081; 49d08-1612-gu-045394; 49d08-2001-gu-001394; 49d08-1610-gu-038294; 53c01-1712-gu-000169; 02d03-2201-gu-000005; 29d01-2308-gu-000140; 53c01-1804-gu-000054; 02d03-2108-gu-000200; 18c01-1606-gu-000059
67,Order Approving Biennial Accounting,504,50c01-1610-gu-000050; 41d01-1610-gu-000159; 02d03-1808-gu-000241; 02d02-1604-gu-000079; 02d01-1701-gu-000009; 02d03-1808-gu-000227; 53c01-1701-gu-000015; 49d08-1608-gu-028488; 50c01-1704-gu-000014; 02d02-1601-gu-000004


In [25]:
search_keyword(['(deny|reject)'], persist=True)

deny_or_reject


,docket_event_description,count,case_examples
122,Order Denying,123,49d08-1808-gu-034594; 31c01-1712-gu-000068; 49d08-1902-gu-005322; 45d02-1905-gu-000146; 41d01-2103-gu-000038; 49d08-1701-gu-002780; 49d08-1702-gu-006906; 49d08-1707-gu-029661; 53c01-1605-gu-000065; 88c01-0505-gu-021
133,Order Denying Petition to Establish Guardianship,91,49d08-1711-gu-044081; 53c01-2308-gu-000129; 03c01-2208-gu-003851; 37c01-1803-gu-000165; 41d01-1909-gu-000171; 49d08-1602-gu-005294; 34d01-2104-gu-000044; 10c01-2012-gu-000120; 41d01-1712-gu-000191; 53c01-2107-gu-000097
171,Order Denying Motion for Continuance,44,41d01-2305-gu-000063; 41d01-1904-gu-000076; 29d05-1710-gu-000206; 49d08-1606-gu-023191; 49d08-1805-gu-016836; 59c01-1710-gu-000057; 49d08-2206-gu-018635; 49d08-1602-gu-005294; 09c01-1608-gu-000044; 41d01-2103-gu-000038
268,Order Denying Motion for Fee Waiver,9,64d02-1705-gu-004286; 52d01-1604-gu-000003; 79c01-1903-gu-000031; 53c01-2112-gu-000180; 34d01-2211-gu-000128; 64d01-2203-gu-002220; 28c01-1609-gu-000026
296,Order Denying Motion to Correct Error,6,49d08-2009-gu-033922; 49d08-1706-gu-023009; 49d08-1805-gu-016836; 27d03-1604-gu-000013; 49d08-1906-gu-022867
297,Order Denying Motion to Dismiss,6,55d01-1610-gu-000096; 49d08-2208-gu-027118; 49d08-1603-gu-008392; 49d08-2102-gu-005794
342,Order Denying Motion for Change of Judge,4,49d04-1605-gu-015747; 29d01-2106-gu-000108; 49d08-1906-gu-022867; 49d08-2210-gu-037603
343,Order Denying Motion to Consolidate,4,49d08-2201-gu-002164; 49d08-1711-gu-043747; 49d08-1811-gu-044220; 49d08-2007-gu-024495
369,Order Denying Motion to Terminate Guardianship,3,45c01-1807-gu-000153; 45d02-1807-gu-000138; 31c01-1611-gu-000055
374,Order Denying Motion for Hearing,3,64d01-1704-gu-003500; 29d01-2106-gu-000108; 49d08-2109-gu-032952


In [26]:
search_keyword(['(guardian|conservator)'], persist=True)

guardian_or_conservator


,docket_event_description,count,case_examples
5,Petition to Establish Guardianship Filed,7906,29d02-2304-gu-000062; 20d01-1603-gu-000024; 02d03-1710-gu-000189; 49d08-2304-gu-014590; 79c01-1903-gu-000031; 02d01-1604-gu-000088; 10c04-1703-gu-000027; 49d08-2007-gu-024696; 71j01-1702-gu-000026; 82d04-1604-gu-000244
7,Letters of Guardianship Issued,7439,53c01-2303-gu-000024; 40d01-1612-gu-000049; 41d01-1708-gu-000133; 45d02-1806-gu-000105; 49d08-2305-gu-020002; 49d08-1809-gu-038559; 89d02-1906-gu-000072; 49d08-1601-gu-003331; 49d08-1701-gu-002485; 24c01-1603-gu-000172
8,Hearing on Petition to Establish Guardianship,6716,49d08-1608-gu-028455; 49d08-2301-gu-003244; 49d08-2312-gu-047284; 49d08-2104-gu-011148; 02d03-1702-gu-000025; 49d08-2003-gu-010981; 53c01-1801-gu-000011; 49d08-2108-gu-026998; 49d08-2006-gu-020441; 49d08-2204-gu-012462
9,Order Granting Petition to Establish Guardianship,6502,71j01-1701-gu-000013; 49d08-1701-gu-004003; 27d03-1811-gu-000113; 49d08-2111-gu-038255; 02d03-2106-gu-000131; 02d03-1604-gu-000082; 49d08-1706-gu-023555; 10c01-1605-gu-000083; 45c01-1809-gu-000219; 41d01-2008-gu-000131
12,Guardianship Information Sheet Filed,4825,47c01-2205-gu-000045; 49d08-2103-gu-009550; 02d03-2105-gu-000112; 49d08-2112-gu-042498; 20d01-1901-gu-000001; 20d01-2108-gu-000137; 32d01-1908-gu-000092; 18c01-2109-gu-000124; 48c01-1903-gu-000138; 49d08-2203-gu-007764
28,Petition for Emergency/Temporary Guardianship Filed,1976,32d01-2009-gu-000083; 20d01-1603-gu-000016; 49d08-2111-gu-038003; 48c06-2310-gu-000532; 02d03-1701-gu-000002; 18c01-1807-gu-000098; 22c01-2312-gu-000112; 45d02-1807-gu-000130; 02d09-1603-gu-000059; 50c01-1809-gu-000063
31,Order Granting Temporary Guardianship,1829,41d01-2011-gu-000165; 49d08-1906-gu-024381; 49d08-2012-gu-045186; 45d05-2206-gu-000170; 02d03-2003-gu-000060; 49d08-1609-gu-033799; 79c01-1703-gu-000038; 49d08-2305-gu-020263; 53c01-2006-gu-000066; 53c01-2107-gu-000089
32,Order Granting Motion to Terminate Guardianship,1793,60c01-1711-gu-000029; 49d08-2001-gu-002027; 49d08-2102-gu-004946; 31c01-1604-gu-000015; 49d08-1609-gu-033865; 41d01-1707-gu-000109; 49d08-1602-gu-006113; 52d01-2002-gu-000006; 49d08-1702-gu-006582; 41d01-1611-gu-000174
45,Hearing on Petition for Emergency Temporary Guardianship,1167,49d08-1906-gu-022745; 49d08-1606-gu-021206; 49d08-2401-gu-003560; 28c01-2212-gu-000038; 89d02-1810-gu-000103; 49d08-2006-gu-021170; 49d08-1605-gu-018483; 49d08-2203-gu-007111; 49d08-2005-gu-016221; 02d03-2008-gu-000193
47,Instruction Executed - Executor/Personal Rep/ Guardian,1062,48c06-1810-gu-000527; 49d08-1606-gu-023530; 32d01-2012-gu-000108; 48c01-1810-gu-000525; 49d08-2010-gu-036121; 10c04-1603-gu-000038; 89c01-2202-gu-000030; 49d08-1712-gu-046430; 49d08-1706-gu-023555; 49d08-1606-gu-022843


In [27]:
search_keyword(['(guardian|conservator|gal|casa)','(grant|accept|deny|reject|approv|order)'], persist=True)

guardian_or_conservator_or_gal_or_casa__and__grant_or_accept_or_deny_or_reject_or_approv_or_order


,docket_event_description,count,case_examples
9,Order Granting Petition to Establish Guardianship,6502,71j01-1701-gu-000013; 49d08-1701-gu-004003; 27d03-1811-gu-000113; 49d08-2111-gu-038255; 02d03-2106-gu-000131; 02d03-1604-gu-000082; 49d08-1706-gu-023555; 10c01-1605-gu-000083; 45c01-1809-gu-000219; 41d01-2008-gu-000131
31,Order Granting Temporary Guardianship,1829,41d01-2011-gu-000165; 49d08-1906-gu-024381; 49d08-2012-gu-045186; 45d05-2206-gu-000170; 02d03-2003-gu-000060; 49d08-1609-gu-033799; 79c01-1703-gu-000038; 49d08-2305-gu-020263; 53c01-2006-gu-000066; 53c01-2107-gu-000089
32,Order Granting Motion to Terminate Guardianship,1793,60c01-1711-gu-000029; 49d08-2001-gu-002027; 49d08-2102-gu-004946; 31c01-1604-gu-000015; 49d08-1609-gu-033865; 41d01-1707-gu-000109; 49d08-1602-gu-006113; 52d01-2002-gu-000006; 49d08-1702-gu-006582; 41d01-1611-gu-000174
103,Order Granting Extension of Guardianship,185,45c01-2012-gu-000281; 02d03-2306-gu-000174; 49d08-2111-gu-038917; 49d08-1901-gu-003694; 49d08-2003-gu-011610; 49d08-1907-gu-029342; 49d08-2305-gu-019421; 49d08-2003-gu-012025; 49d08-1711-gu-041172
133,Order Denying Petition to Establish Guardianship,91,49d08-1711-gu-044081; 53c01-2308-gu-000129; 03c01-2208-gu-003851; 37c01-1803-gu-000165; 41d01-1909-gu-000171; 49d08-1602-gu-005294; 34d01-2104-gu-000044; 10c01-2012-gu-000120; 41d01-1712-gu-000191; 53c01-2107-gu-000097
257,Order Granting Motion to Remove Guardian,10,54d01-1806-gu-000021; 24c01-1809-gu-000674; 49d08-1804-gu-013870; 49d08-1603-gu-012441; 33c02-1607-gu-000028; 10c01-1612-gu-000191
369,Order Denying Motion to Terminate Guardianship,3,45c01-1807-gu-000153; 45d02-1807-gu-000138; 31c01-1611-gu-000055
408,Order Denying Motion to Remove Guardian,2,27d03-1706-gu-000051; 49d08-2008-gu-028430
412,Order Denying Extension of Guardianship,2,43d01-1809-gu-000071; 12c01-2302-gu-000007


In [28]:
search_keyword(['petition'], persist=True)

petition


,docket_event_description,count,case_examples
5,Petition to Establish Guardianship Filed,7906,29d02-2304-gu-000062; 20d01-1603-gu-000024; 02d03-1710-gu-000189; 49d08-2304-gu-014590; 79c01-1903-gu-000031; 02d01-1604-gu-000088; 10c04-1703-gu-000027; 49d08-2007-gu-024696; 71j01-1702-gu-000026; 82d04-1604-gu-000244
8,Hearing on Petition to Establish Guardianship,6716,49d08-1608-gu-028455; 49d08-2301-gu-003244; 49d08-2312-gu-047284; 49d08-2104-gu-011148; 02d03-1702-gu-000025; 49d08-2003-gu-010981; 53c01-1801-gu-000011; 49d08-2108-gu-026998; 49d08-2006-gu-020441; 49d08-2204-gu-012462
9,Order Granting Petition to Establish Guardianship,6502,71j01-1701-gu-000013; 49d08-1701-gu-004003; 27d03-1811-gu-000113; 49d08-2111-gu-038255; 02d03-2106-gu-000131; 02d03-1604-gu-000082; 49d08-1706-gu-023555; 10c01-1605-gu-000083; 45c01-1809-gu-000219; 41d01-2008-gu-000131
25,Petition Filed,2205,73d01-1801-gu-000001; 73d01-1605-gu-000036; 45d11-1806-gu-000118; 84d01-1704-gu-002618; 91c01-1903-gu-000009; 18c01-2103-gu-000026; 45d11-2211-gu-000318; 59c01-2008-gu-000055; 37c01-1608-gu-000646; 06d01-1705-gu-000021
28,Petition for Emergency/Temporary Guardianship Filed,1976,32d01-2009-gu-000083; 20d01-1603-gu-000016; 49d08-2111-gu-038003; 48c06-2310-gu-000532; 02d03-1701-gu-000002; 18c01-1807-gu-000098; 22c01-2312-gu-000112; 45d02-1807-gu-000130; 02d09-1603-gu-000059; 50c01-1809-gu-000063
45,Hearing on Petition for Emergency Temporary Guardianship,1167,49d08-1906-gu-022745; 49d08-1606-gu-021206; 49d08-2401-gu-003560; 28c01-2212-gu-000038; 89d02-1810-gu-000103; 49d08-2006-gu-021170; 49d08-1605-gu-018483; 49d08-2203-gu-007111; 49d08-2005-gu-016221; 02d03-2008-gu-000193
49,Petition to Appoint GAL/CASA Filed,952,02d03-2002-gu-000021; 02d03-1909-gu-000172; 02d03-2008-gu-000187; 02d03-2110-gu-000271; 20c01-2306-gu-000084; 02d03-2103-gu-000079; 02d03-2208-gu-000237; 02d03-2007-gu-000149; 02d03-2308-gu-000256; 02d03-2303-gu-000068
133,Order Denying Petition to Establish Guardianship,91,49d08-1711-gu-044081; 53c01-2308-gu-000129; 03c01-2208-gu-003851; 37c01-1803-gu-000165; 41d01-1909-gu-000171; 49d08-1602-gu-005294; 34d01-2104-gu-000044; 10c01-2012-gu-000120; 41d01-1712-gu-000191; 53c01-2107-gu-000097
151,Petition for Extension of Temporary Guardianship,63,45c01-1809-gu-000215; 49d08-2209-gu-031315; 18c01-2004-gu-000035; 49d08-2212-gu-043616; 49d08-2211-gu-038314; 49d08-2006-gu-018216; 49d08-2212-gu-043389; 49d08-2008-gu-028002; 18c01-2102-gu-000017; 49d08-2007-gu-023013
152,Petition for Appointment of Personal Representative Filed,61,18c01-2301-gu-000008; 49d08-1701-gu-002976; 41d01-1805-gu-000088; 41d01-2104-gu-000050; 53c01-1703-gu-000033; 02d03-1810-gu-000288; 41d01-2106-gu-000091; 41d01-2305-gu-000063; 02d03-2211-gu-000314; 27d03-1804-gu-000027


In [29]:
search_keyword(['temporary'], persist=True)

temporary


,docket_event_description,count,case_examples
28,Petition for Emergency/Temporary Guardianship Filed,1976,32d01-2009-gu-000083; 20d01-1603-gu-000016; 49d08-2111-gu-038003; 48c06-2310-gu-000532; 02d03-1701-gu-000002; 18c01-1807-gu-000098; 22c01-2312-gu-000112; 45d02-1807-gu-000130; 02d09-1603-gu-000059; 50c01-1809-gu-000063
31,Order Granting Temporary Guardianship,1829,41d01-2011-gu-000165; 49d08-1906-gu-024381; 49d08-2012-gu-045186; 45d05-2206-gu-000170; 02d03-2003-gu-000060; 49d08-1609-gu-033799; 79c01-1703-gu-000038; 49d08-2305-gu-020263; 53c01-2006-gu-000066; 53c01-2107-gu-000089
45,Hearing on Petition for Emergency Temporary Guardianship,1167,49d08-1906-gu-022745; 49d08-1606-gu-021206; 49d08-2401-gu-003560; 28c01-2212-gu-000038; 89d02-1810-gu-000103; 49d08-2006-gu-021170; 49d08-1605-gu-018483; 49d08-2203-gu-007111; 49d08-2005-gu-016221; 02d03-2008-gu-000193
132,Expiration of Temporary Guardianship,93,49d08-2212-gu-042569; 30d01-2202-gu-000018; 49d08-2206-gu-018684; 49d08-2207-gu-022170; 49d08-2308-gu-030643; 20d01-2108-gu-000137; 49d08-2110-gu-033807; 48c01-2207-gu-000391; 49d08-2207-gu-025703
151,Petition for Extension of Temporary Guardianship,63,45c01-1809-gu-000215; 49d08-2209-gu-031315; 18c01-2004-gu-000035; 49d08-2212-gu-043616; 49d08-2211-gu-038314; 49d08-2006-gu-018216; 49d08-2212-gu-043389; 49d08-2008-gu-028002; 18c01-2102-gu-000017; 49d08-2007-gu-023013
212,Temporary Appearance Filed,21,49d08-1812-gu-049281; 49d08-2204-gu-012773; 59c01-2112-gu-000110; 45c01-1802-gu-000026; 73d01-1605-gu-000035; 49d08-2202-gu-006486; 49d08-2210-gu-037603; 49d08-2202-gu-005016; 49d08-1804-gu-013765
294,Motion for Temporary Restraining Order Filed,6,49d08-1902-gu-007303; 49d08-1905-gu-021122; 49d08-2305-gu-017882; 79c01-2010-gu-000083; 49d08-2006-gu-021479
443,Temporary/Limited Scope Appearance Filed,1,84d02-2304-gu-002357
496,Order Granting Motion for Temporary Restraining Order,1,49d08-2006-gu-021479
508,Petition for Temporary Mental Health Commitment Filed,1,53c01-2307-gu-000117


In [30]:
search_keyword(['(death|decease|dead|demise|died|funeral)'], persist=True)

death_or_decease_or_dead_or_demise_or_died_or_funeral


,docket_event_description,count,case_examples


In [31]:
frequency_table[frequency_table.docket_event_description.str.contains('Notice Filed')]

,docket_event_description,count,case_examples
16,Notice Filed,3238,02d03-2202-gu-000036; 46d01-1912-gu-000119; 02d03-2301-gu-000027; 12d01-2111-gu-000061; 18c01-2004-gu-000038; 02d03-1704-gu-000086; 49d08-1707-gu-028217; 65d01-2304-gu-000007; 65c01-2209-gu-000022; 20d01-2208-gu-000110
73,Waiver of Notice Filed,416,18c01-2108-gu-000106; 33c02-2103-gu-000015; 10c04-1702-gu-000014; 21d01-1702-gu-000004; 49d08-2101-gu-000146; 73c01-2306-gu-000019; 20d01-2109-gu-000158; 76c01-1609-gu-000031; 49d08-1806-gu-021835; 13c01-1703-gu-000006
318,Proof of Publication of Notice Filed,5,18c01-1811-gu-000140; 46c01-2006-gu-000050; 20d01-1609-gu-000123; 22c01-1703-gu-000025
328,Lis Pendens Notice Filed,4,80c01-1908-gu-000022; 49d08-1812-gu-048993; 49d08-2102-gu-005294; 49d08-1711-gu-042819


In [32]:
search_keyword(['dismiss'], persist=True)

dismiss


,docket_event_description,count,case_examples
64,Motion to Dismiss Filed,598,08c01-1705-gu-000008; 49d08-1801-gu-003569; 49d08-1801-gu-001670; 02d03-2012-gu-000277; 49d08-1604-gu-014850; 49d08-2206-gu-020761; 49d08-2301-gu-000392; 49d08-1801-gu-002490; 28c01-2107-gu-000026; 46c01-2003-gu-000033
66,Order of Dismissal,564,02d03-2304-gu-000112; 47c01-2304-gu-000031; 49d08-1906-gu-024956; 49d08-2008-gu-027447; 20c01-1711-gu-000139; 53c01-2101-gu-000005; 53c01-2208-gu-000100; 49d08-2305-gu-019167; 49d08-2104-gu-012599; 02d03-2111-gu-000312
83,Order Granting Motion to Dismiss,314,49d08-1903-gu-008619; 02d02-1606-gu-000130; 49d08-2305-gu-017944; 13c01-2201-gu-000002; 49d08-1608-gu-030790; 41d01-1605-gu-000074; 55d01-1608-gu-000072; 49d08-2208-gu-029704; 28c01-1706-gu-000026; 02d03-1809-gu-000250
142,Hearing on Motion for 41(E) Dismissal,86,49d08-2103-gu-007766; 49d08-1809-gu-037172; 27d03-1611-gu-000082; 02d03-2205-gu-000127; 49d08-1908-gu-033951; 49d08-2104-gu-012788; 49d08-2007-gu-022210; 49d08-2006-gu-021147; 49d08-1908-gu-034833; 02d03-2104-gu-000089
183,Hearing on Motion for 41E Dismissal,38,55d01-1610-gu-000100; 02d03-1708-gu-000159; 49d08-1803-gu-009095; 50c01-1710-gu-000032; 20d01-1806-gu-000129; 18c01-1610-gu-000112; 27d03-1803-gu-000025; 45d11-1803-gu-000040
231,Dismissed,15,48c01-2312-gu-000599; 53c07-1607-gu-000089; 48c06-2008-gu-000351; 18c01-1801-gu-000011; 48c01-2305-gu-000259; 28c01-2207-gu-000025; 52c01-1812-gu-000017; 18c01-2204-gu-000084; 48c06-1710-gu-000486; 52c01-2107-gu-000036
236,Order of 41E Dismissal,14,27d03-1608-gu-000050; 18c01-1605-gu-000053; 71d05-1608-gu-000163; 20d01-1608-gu-000115; 37c01-1802-gu-000134; 59c01-2008-gu-000055; 55d01-1610-gu-000100; 71d05-1608-gu-000164
250,Notice of Dismissal Filed,11,49d08-2009-gu-032285; 28c01-2301-gu-000001; 49d08-1903-gu-011770; 49d08-2012-gu-042580; 49d08-2009-gu-030536; 49d08-2203-gu-007013; 49d08-1610-gu-036977
264,Hearing on Motion to Dismiss,9,53c01-2105-gu-000062; 34d01-1911-gu-000164; 45d02-1905-gu-000146; 49d05-1904-gu-017471; 89c01-2111-gu-000140; 71c01-1610-gu-000222
273,Case Dismissed,8,52c01-2102-gu-000008; 48c06-1801-gu-000035; 45c01-1805-gu-000078; 49d08-1710-gu-038414; 50c01-2103-gu-000011; 48c03-1911-gu-000643; 34d01-1911-gu-000164; 10c01-1612-gu-000183


### Missing Outcome Frequency Table

### From Just City EDA notebook

In [ ]:
ddf = pd.read_csv('/Users/ryancarroll/Downloads/indiana_guardianship_dockets.csv')
ddf['granted_substrings'] = ddf.docket_event_comment.str.lower().str.findall('\w+ \w+ granted \w+ \w+')
found_substr = ~ddf.granted_substrings.isnull() & (ddf.granted_substrings.astype(str) != '[]')
ddf.loc[found_substr, 'granted_substring'] = ddf[found_substr].granted_substrings.astype(str).str.replace('[', '').str.replace(']', '')
ddf.granted_substring.unique()
ddf[~ddf.granted_substrings.isnull() & (ddf.granted_substrings.astype(str) != '[]')]